In [1]:
mErde, mOzean, RErde, mMond, rMondBahn, TMondBahn, G = 5.9721986*10**24, 0.0014*10**24, 6.3675*10**6, 7.3459*10**22, 3.836*10**8, 27.32166140*24*3600, 6.67430*10**(-11)
TErdRotation = 86164.100        # s           : Heutige Periodendauer der intrinsischen Erdrotation
tau = 0.0021                    # s/100a      : Zunahme der Tageslänge auf der Erde in 100 Jahren

import numpy as np
import matplotlib.pyplot as plt
import math
import sympy
from scipy.constants import pi
import scipy
from scipy.integrate import solve_ivp
from scipy.integrate._ivp.ivp import OdeResult # :) https://github.com/scipy/scipy/blob/main/scipy/integrate/_ivp/ivp.py
from matplotlib.animation import FuncAnimation
from IPython.display import Image #to display animations

# Beautiful plots
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "Computer Modern Roman"

# Useful StackPosts
# https://stackoverflow.com/questions/59634279/solve-ivp-error-required-step-size-is-less-than-spacing-between-numbers

# Executing everything all the time takes too long
fastExecution = True

In [2]:
def baryzentrum(m1, m2, r): return r*m2/(m1+m2) # TODO Remove, redundant

def iv_stable_orbit_4body():
    '''Initial conditions for the Earth-Moon-Tides system in a stable orbit'''
    # Each of the tides gets half of the oceans mass 
    mass = [mErde, mMond, mOzean/2, mOzean/2]
    
    # We want the center of mass to be at the origin, the tides count towards the earth,
    # but weigh so little that the center of doesn't really change
    abstand_baryzentrum_erde = baryzentrum(mErde + mOzean, mMond, rMondBahn)
    x0_Erde = [-abstand_baryzentrum_erde, 0]
    x0_Mond = [rMondBahn-abstand_baryzentrum_erde, 0]

    # We choose the starting velocities such that the center of mass is at rest
    vy0_Erde = 2*pi*abstand_baryzentrum_erde/TMondBahn
    vy0_Mond = 2*pi*(rMondBahn-abstand_baryzentrum_erde)/TMondBahn

    # The moon starts in positive, the earth in negative y-direction
    v0_Erde = [0, -vy0_Erde]
    v0_Mond = [0, vy0_Mond]

    # The tides start on the same axis as the moon on opposite sides of the earth
    phi_1 = 0
    phi_2 = pi

    # The tides start with the same velocity as the moon
    omega_1 = 2*pi/TMondBahn
    omega_2 = 2*pi/TMondBahn

    return [x0_Erde, x0_Mond, phi_1, phi_2, v0_Erde, v0_Mond, omega_1, omega_2, mass]

In [3]:
k = 2*10**(-12)

def tides_acceleration_spin(x_E, y_E, vx_Ed, vy_Ed, x_M, y_M, phi_i, omega_i, m_M):
    '''Calculates the angular acceleration of a tide, using the equation above'''
    vx_id = -G*(m_M*((y_M - (y_E + RErde*np.sin(phi_i)))*np.cos(phi_i) - (x_M - (x_E + RErde*np.cos(phi_i)))*np.sin(phi_i))) / ((x_M - (x_E + RErde*np.cos(phi_i)))**2 + (y_M - (y_E + RErde*np.sin(phi_i)))**2)**(3/2) * np.sin(phi_i) -(RErde*omega_i**2 - (vx_Ed*np.cos(phi_i) + vy_Ed*np.sin(phi_i)))*np.cos(phi_i) - omega_i*RErde*np.sin(phi_i)
    vy_id = G*(m_M*((y_M - (y_E + RErde*np.sin(phi_i)))*np.cos(phi_i) - (x_M - (x_E + RErde*np.cos(phi_i)))*np.sin(phi_i))) / ((x_M - (x_E + RErde*np.cos(phi_i)))**2 + (y_M - (y_E + RErde*np.sin(phi_i)))**2)**(3/2) * np.cos(phi_i) -(RErde*omega_i**2 - (vx_Ed*np.cos(phi_i) + vy_Ed*np.sin(phi_i)))*np.sin(phi_i) + omega_i*RErde*np.cos(phi_i)
    alpha_i = (-(vx_id - vx_Ed)*np.sin(phi_i) + (vy_id - vy_Ed)*np.cos(phi_i))/RErde

    return alpha_i

def eq_motion_4body_spin(t, state, mass):
    x_E, y_E, x_M, y_M, phi_1, phi_2, phi_E, vx_E, vy_E, vx_M, vy_M, omega_1, omega_2, omega_E = state
    m_E, m_M, m_1, m_2 = mass
    
    # Variables as defined in the explanation above
    mu_x = m_E + m_1*np.cos(phi_1)**2 + m_2*np.cos(phi_2)**2
    mu_y = m_E + m_1*np.sin(phi_1)**2 + m_2*np.sin(phi_2)**2
    a = (G*m_M*m_E)/((x_M - x_E)**2 + (y_M - y_E)**2)**(3/2)
    b = m_1*np.sin(phi_1)*np.cos(phi_1) + m_2*np.sin(phi_2)*np.cos(phi_2)
    c1 = m_1*RErde*omega_1**2 + (G*m_M*m_1*((x_M - (x_E + RErde*np.cos(phi_1)))*np.cos(phi_1) + (y_M - (y_E + RErde*np.sin(phi_1)))))/((x_M - (x_E + RErde*np.cos(phi_1)))**2 + (y_M - (y_E + RErde*np.sin(phi_1)))**2)**(3/2)
    c2 = m_2*RErde*omega_2**2 + (G*m_M*m_2*((x_M - (x_E + RErde*np.cos(phi_2)))*np.cos(phi_2) + (y_M - (y_E + RErde*np.sin(phi_2)))))/((x_M - (x_E + RErde*np.cos(phi_2)))**2 + (y_M - (y_E + RErde*np.sin(phi_2)))**2)**(3/2)

    # The earth's acceleration:    
    ax_E = 1/(mu_x - b**2/mu_y) * (a*(x_M - x_E) -b/mu_y*(a*(y_M - y_E) + c1*np.sin(phi_1) + c2*np.sin(phi_2)) + c1*np.cos(phi_1) + c2*np.cos(phi_2))
    ay_E = 1/mu_y*(a*(y_M - y_E) - b*ax_E + c1*np.sin(phi_1) + c2*np.sin(phi_2))

    # The moon's acceleration:
    ax_M = G*((m_E*(x_E - x_M))/((x_E - x_M)**2 + (y_E - y_M)**2)**(3/2) + (m_1*(x_E + RErde*np.cos(phi_1) - x_M))/((x_E + RErde*np.cos(phi_1) - x_M)**2 + (y_E + RErde*np.sin(phi_1) - y_M)**2)**(3/2) + (m_2*(x_E + RErde*np.cos(phi_2) - x_M))/((x_E + RErde*np.cos(phi_2) - x_M)**2 + (y_E + RErde*np.sin(phi_2) - y_M)**2)**(3/2))
    vy_Md = G*((m_E*(y_E - y_M))/((x_E - x_M)**2 + (y_E - y_M)**2)**(3/2) + (m_1*(y_E + RErde*np.cos(phi_1) - y_M))/((x_E + RErde*np.cos(phi_1) - x_M)**2 + (y_E + RErde*np.sin(phi_1) - y_M)**2)**(3/2) + (m_2*(y_E + RErde*np.cos(phi_2) - y_M))/((x_E + RErde*np.cos(phi_2) - x_M)**2 + (y_E + RErde*np.sin(phi_2) - y_M)**2)**(3/2))

    # Tides angular acceleration:
    alpha_1 = tides_acceleration_spin(x_E, y_E, ax_E, ay_E, x_M, y_M, phi_1, omega_1, m_M)
    alpha_2 = tides_acceleration_spin(x_E, y_E, ax_E, ay_E, x_M, y_M, phi_2, omega_2, m_M)

    # The earth's angular acceleration:
    alpha_E = -(5*k*RErde)/(2*m_E) * (m_1*omega_1**2 + m_2*omega_2**2)
    # alpha_E = 0
    return[vx_E, vy_E, vx_M, vy_M, omega_1, omega_2, omega_E, ax_E, ay_E, ax_M, vy_Md, alpha_1, alpha_2, alpha_E]

def iv_stable_orbit_4body_spin():
    '''Initial conditions for the Earth-Moon-Tides system in a stable orbit'''
    x0_Erde, x0_Mond, phi_1, phi_2, v0_Erde, v0_Mond, omega_1, omega_2, mass = iv_stable_orbit_4body()
    phi0_E = 0
    omega0_E = 7.2921159*10**(-5) # Winkelgeschwindigkeit der Erde von Wikipedia

    return [x0_Erde, x0_Mond, phi_1, phi_2, phi0_E, v0_Erde, v0_Mond, omega_1, omega_2, omega0_E, mass]

def four_body_problem_spin(pos_body_1: list, pos_body_2: list, phi_1: float, phi_2: float, phi_E:float, vel_body_1: list, vel_body_2: list, omega_1: float, omega_2: float, omega_E: float, mass: list, t_max: float, steps=1, method='RK45', t_start=0):
    mass = [mErde, mMond, mOzean/2, mOzean/2]
    solution = solve_ivp(fun=eq_motion_4body_spin, t_span=[t_start, t_max], y0=[*pos_body_1, *pos_body_2, phi_1, phi_2, phi_E, *vel_body_1, *vel_body_2, omega_1, omega_2, omega_E], args=(mass,))
    x_E, y_E, x_M, y_M, phi_1, phi_2, phi_E, vx_E, vy_E, vx_M, vy_M, omega_1, omega_2, omega_E = solution.y
    return [solution.t, x_E, y_E, x_M, y_M, phi_1, phi_2, phi_E]

t, x_E, y_E, x_M, y_M, phi_1, phi_2, phi_E = four_body_problem_spin(*iv_stable_orbit_4body_spin(), TMondBahn)

KeyboardInterrupt: 

In [ ]:
x_1, y_1 = x_E + RErde*np.cos(phi_1), y_E + RErde*np.sin(phi_1)
x_2, y_2 = x_E + RErde*np.cos(phi_2), y_E + RErde*np.sin(phi_2)

plt.rc('axes', axisbelow=True)
plt.figure(figsize=(5, 5))
plt.grid('minor', 'minor', linestyle='-', linewidth=0.2)
plt.grid('major', 'major', linestyle='-', linewidth=0.8)
plt.minorticks_on()
plt.xlabel("x")
plt.ylabel("y")

plt.plot(x_E, y_E, 'k', linewidth=1, label='Erde')
plt.plot(x_M/10, y_M/10, 'r', linewidth=1, label='Mond (/10)')
plt.plot(x_1, y_1, 'b', linewidth=1, label='Flutberg 1')
plt.plot(x_2, y_2, 'b', linewidth=1, label='Flutberg 2')
plt.plot(x_E + RErde/2*np.cos(phi_E), y_E + RErde/2*np.sin(phi_E), 'g', linewidth=1, label='Äquator')

plt.legend()
plt.show()
plt.close()

# Plot a 3d phase space diagram
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.plot(x_E, y_E, t, 'k', label='Erde')
ax.plot(x_M/10, y_M/10, t, 'r', label='Mond')
ax.plot(x_1, y_1, t, 'b', label='Flutberg 1')
ax.plot(x_2, y_2, t, 'b', label='Flutberg 2')
ax.plot(x_E + RErde/2*np.cos(phi_E), y_E + RErde/2*np.sin(phi_E), t, 'g', label='Äquator')
ax.set_xlabel("$x$ / m")
ax.set_ylabel("$y$ / m")
ax.set_zlabel("$t$ / s")
ax.legend()
plt.show()
plt.close()

# Plot the angles of the tides over time
plt.rc('axes', axisbelow=True)
plt.figure(figsize=(5, 5))
plt.grid('minor', 'minor', linestyle='-', linewidth=0.2)
plt.grid('major', 'major', linestyle='-', linewidth=0.8)
plt.minorticks_on()
plt.xlabel("$t$ / s")
plt.ylabel("$\phi$ / rad")

plt.plot(t, phi_1 , 'k', linewidth=1, label='Flutberg 1')
plt.plot(t, phi_2, 'r', linewidth=1, label='Flutberg 2')
plt.plot(t, phi_E, 'b', linewidth=1, label='Äquator')

plt.legend()
plt.show()
plt.close()